<a href="https://colab.research.google.com/github/yeonghun00/stock_public/blob/main/stock_screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import requests
import datetime

In [2]:
today = datetime.date.today().strftime('%Y%m%d')

In [3]:
data = {
    'mktId': 'KSQ',
    'trdDd': today,
    'money': '1',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03901',
}
gen_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_key = requests.post(gen_url, data=data)

In [4]:
down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
r = requests.post(down_url, data={'code':gen_key.text})
r.encoding = 'EUC-KR'

In [5]:
from io import StringIO

df = pd.read_csv(StringIO(r.text))

In [6]:
df

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
0,060310,3S,KOSDAQ,기계·장비,4115,210,5.38,190407275995
1,054620,APS홀딩스,KOSDAQ,금융,17400,-150,-0.85,354859445400
2,265520,AP시스템,KOSDAQ,반도체,26950,-750,-2.71,390242117650
3,211270,AP위성,KOSDAQ,통신장비,17900,200,1.13,269973241600
4,013720,CBI,KOSDAQ,운송장비·부품,3540,260,7.93,179470123500
...,...,...,...,...,...,...,...,...
1508,024060,흥구석유,KOSDAQ,유통,7850,430,5.80,117750000000
1509,010240,흥국,KOSDAQ,기계·장비,9550,420,4.60,117681746800
1510,189980,흥국에프엔비,KOSDAQ,음식료·담배,3310,125,3.92,127061390370
1511,037440,희림,KOSDAQ,기타서비스,7090,440,6.62,98710347750


In [7]:
%%timeit
df.loc[df['시가총액'].between(30000000000,70000000000), '종목명']

The slowest run took 34.06 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 5: 568 µs per loop


In [8]:
%%timeit
df[df['시가총액'].between(30000000000,70000000000)]['종목명']

1000 loops, best of 5: 912 µs per loop


In [9]:
boundary = df.loc[df['시가총액'].between(30000000000,70000000000)]
boundary

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
10,056000,COWON,KOSDAQ,디지털컨텐츠,2860,0,0.00,56399843500
12,083660,CSA 코스믹,KOSDAQ,화학,1525,15,0.99,57849523850
20,050120,ES큐브,KOSDAQ,유통,4700,-40,-0.84,63751204200
22,900290,GRT,KOSDAQ,금융,951,0,0.00,64073625000
35,099520,ITX-AI,KOSDAQ,통신장비,1285,0,0.00,37652662655
...,...,...,...,...,...,...,...,...
1461,079170,한창산업,KOSDAQ,금속,9610,190,2.02,49972000000
1464,002680,한탑,KOSDAQ,음식료·담배,1400,-10,-0.71,39577983200
1473,900270,헝셩그룹,KOSDAQ,금융,508,6,1.20,40640000000
1486,011080,형지I&C,KOSDAQ,섬유·의류,1480,110,8.03,57654388640


In [10]:
!pip install OpenDartReader

In [11]:
import OpenDartReader

api_key = '4b0512c473e52e1cf59f8ce8380b2b30904f381a'
dart = OpenDartReader(api_key) 

In [50]:
search_start = '2021-04-01'
search_end = '2021-05-01'

In [51]:
dart_df = dart.list(start=search_start, end=search_end, kind='A')
found = list(set(dart_df['stock_code']).intersection(set(boundary['종목코드'])))
out_df = dart_df[dart_df['stock_code'].isin(found)]

In [52]:
out_df

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
6,01139266,로스웰,900260,K,사업보고서 (2020.12),20210430000726,로스웰,20210430,연
7,01160512,헝셩그룹,900270,K,사업보고서 (2020.12),20210430000248,헝셩그룹,20210430,연
41,00814786,스카이이앤엠,131100,K,[기재정정]사업보고서 (2020.06),20210423000574,스카이이앤엠,20210423,연
57,01328639,윙입푸드,900340,K,사업보고서 (2020.12),20210420000469,윙입푸드,20210420,연
58,00200910,아이즈비전,031310,K,[기재정정]사업보고서 (2019.12),20210420000465,아이즈비전,20210420,연
59,00200910,아이즈비전,031310,K,[기재정정]사업보고서 (2018.12),20210420000460,아이즈비전,20210420,연
68,00298687,스포츠서울,039670,K,[기재정정]사업보고서 (2020.03),20210420000215,스포츠서울,20210420,연
1,00361266,COWON,056000,K,[기재정정]사업보고서 (2020.12),20210416000571,COWON,20210416,연
11,00870481,에코캡,128540,K,[기재정정]사업보고서 (2020.12),20210416000238,에코캡,20210416,연
13,00624749,에스피시스템스,317830,K,[기재정정]사업보고서 (2019.12),20210416000155,에스피시스템스,20210416,연


In [53]:
def get_recent_finstate(corp_code='', year=2021):
  quarters = ['11011', '11014', '11012', '11013']
  for q in quarters:
    # '11013'=1분기보고서, '11012'=반기보고서, '11014'=3분기보고서, '11011'=사업보고서
    df = dart.finstate(corp_code, year, reprt_code=q)
    if type(df) != type(None):
      return df[df['fs_nm'] == '연결재무제표']

def get_growth_rate(previous, current):
  return (current-previous)/abs(previous)

In [54]:
df = get_recent_finstate('00402989')
is_df = df.loc[df['sj_nm'] == '손익계산서']

In [46]:
operating_profit = is_df[is_df['account_nm'].isin(['영업이익'])]

In [47]:
current = int(operating_profit['thstrm_amount'].values[0].replace(',',''))

In [48]:
previous = int(operating_profit['frmtrm_amount'].values[0].replace(',',''))

In [55]:
for code in out_df['corp_code']:
  try:
    df = get_recent_finstate(code)
    is_df = df.loc[df['sj_nm'] == '손익계산서']
    operating_profit = is_df[is_df['account_nm'].isin(['영업이익'])]
    current = int(operating_profit['thstrm_amount'].values[0].replace(',',''))
    previous = int(operating_profit['frmtrm_amount'].values[0].replace(',',''))
    print(out_df.loc[out_df['corp_code'] == code, 'corp_name'].values[0])
    print(out_df.loc[out_df['corp_code'] == code, 'rcept_dt'].values[0])
    print(get_growth_rate(previous, current))
    print()
  except:
    print('ERROR', out_df.loc[out_df['corp_code'] == code, 'corp_name'].values[0])
    print()


로스웰
20210430
0.823590080944212

헝셩그룹
20210430
-0.9104158606311058

스카이이앤엠
20210423
0.19975064286302283

윙입푸드
20210420
0.9038904044307687

아이즈비전
20210420
2.2134493830581885

아이즈비전
20210420
2.2134493830581885

스포츠서울
20210420
0.37926051336634825

COWON
20210416
-6.925062909783071

에코캡
20210416
0.9934062214631412

에스피시스템스
20210416
-2.6795533159526124

에스피시스템스
20210416
-2.6795533159526124

풍강
20210414
1.951646433483091

글로벌에스엠
20210409
1.762833471083145

씨엔플러스
20210408
2.268247046800354

CSA 코스믹
20210407
-1.0308588115626063

ERROR 에스아이리소스

루멘스
20210407
-2.9382742707180145

판타지오
20210406
4.687940832444536

씨유메디칼
20210402
0.6791139226158083

강원
20210402
0.9671346635935777

ERROR 원풍물산



'이큐셀'